In [ ]:
import os

openai_api_key = os.environ.get("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("The environment variable OPENAI_API_KEY is not set. Please make sure to set it.")

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


In [ ]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt.tool_validator import ValidationNode

### SUBGRAPH 1

In [ ]:
import pandas as pd
import json
from typing import Annotated, Sequence, TypedDict

# --- Tool Definitions ---

# Define the Imagenow Tool
@tool
def imagenow_tool(invoice_id: str):
    """Check payment status in Imagenow."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Replace with actual file path
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        payment_status = invoice_data.iloc[0]["Status"]
        return {"status": payment_status}
    else:
        return {"error": f"Invoice {invoice_id} not found."}

# Define the Lawson Tool
@tool
def lawson_tool(invoice_id: str):
    """Fetch payment details from Lawson based on the invoice ID."""
    lawson_file_path = "/content/lawson_modified.xlsx"  # Replace with actual file path
    lawson_df = pd.read_excel(lawson_file_path)
    invoice_data = lawson_df[lawson_df["Invoice ID"] == invoice_id]
    if not invoice_data.empty:
        payment_method = invoice_data.iloc[0]["Payment Method"]
        payment_date = invoice_data.iloc[0]["Payment Date"]
        exception_status = invoice_data.iloc[0]["Exception Status"]
        return {
            "payment_method": payment_method,
            "payment_date": payment_date,
            "exception_status": exception_status,
        }
    else:
        return {"error": f"Invoice {invoice_id} not found."}

# Define the Ivalua Tool
@tool
def ivalua_tool(invoice_id: str):
    """Check transmission status in Ivalua based on the invoice ID."""
    ivalua_file_path = "/content/ivalua_dataset.xlsx"  # Replace with actual file path
    ivalua_df = pd.read_excel(ivalua_file_path)
    invoice_row = ivalua_df[ivalua_df["Invoice Number"] == invoice_id]
    if not invoice_row.empty:
        transmission_status = invoice_row.iloc[0]["Transmission Status"]
        exception_status = invoice_row.iloc[0]["Exception status"]
        return {
            "transmission_status": transmission_status,
            "exception_status": exception_status
        }
    else:
        return {"error": f"Invoice {invoice_id} not found in Ivalua."}

# Define the Email Tool
@tool
def email_tool(recipient: str, message: str):
    """Send an email to the vendor or team."""
    return {"email_status": "Sent"}

# List of tools
tools = [imagenow_tool, lawson_tool, ivalua_tool, email_tool]

# --- Agent State Definition ---

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- Tool Node ---

from langchain_core.messages import ToolMessage

# Create a mapping of tools by name for quick lookup
tools_by_name = {tool.name: tool for tool in tools}

def tool_node(state: AgentState):
    outputs = []
    # Process each tool call in the last message
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        # Convert pd.Timestamp to string if necessary
        for key, value in tool_result.items():
            if isinstance(value, pd.Timestamp):
                tool_result[key] = value.isoformat()
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}

# --- Workflow Orchestration ---

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableConfig

# Define the workflow steps prompt
steps_prompt = """
You are an AI orchestrator for the Payment Inquiry workflow. Follow these steps to process the query:

1. Check the payment status in Imagenow using the invoice ID.
2. If the status is "Paid":
   - Fetch payment details (method and date) from Lawson.
   - Respond to the vendor with the payment details.
   - Update the notes in Imagenow.
3. If the status is "Not Paid":
   - Determine whether the PO is a 10-digit or 11-digit number.
   - For an 11-digit PO:
       - Check transmission status in Ivalua.
       - If not transmitted, notify the appropriate person and update the notes.
       - If transmitted, check exception status in Lawson.
   - For a 10-digit PO:
       - Check exception status in Lawson.
       - Notify the appropriate person based on the exception status.
4. At each step, use the appropriate tool and reason about the next action based on the tool's result.
"""

# Initialize the LLM and bind the tools
model1 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools)

def call_model(state: AgentState, config: RunnableConfig):
    system_prompt = SystemMessage(
        content=f"{steps_prompt}\n\nYou are a workflow orchestrator for Payment Inquiry. Decide which tool to use or provide a final answer based on the query."
    )
    # Invoke the model with the system prompt and current message history
    response = model1.invoke([system_prompt] + state["messages"], config)
    # Append the response to the state so subsequent nodes can see it
    state["messages"].append(response)
    return state

def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If the last message contains tool calls, continue to the "tools" node
    if last_message.tool_calls:
        return "tools"
    return "end"

# --- Graph Construction ---

from langgraph.graph import StateGraph, END

# Initialize the StateGraph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entry point
workflow.set_entry_point("agent")

# Add conditional edges based on whether there are pending tool calls
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})

# Add an edge from the tools node back to the agent node
workflow.add_edge("tools", "agent")

# Compile the workflow to produce graph1
graph1 = workflow.compile()


In [ ]:
import time

# Helper function for formatting the output stream
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

# Define user query
inputs = {"messages": [("user", "Check the payment status of invoice INV-0003.PO number is 77649657916")]}

# Start timing the execution
start_time = time.time()

# Execute the workflow and print the output stream
print_stream(graph1.stream(inputs, stream_mode="values"))

# End timing and calculate the execution time
end_time = time.time()
execution_time = end_time - start_time

# Print the execution time
print(f"Execution Time: {execution_time} seconds")

================================ Human Message =================================

Check the payment status of invoice INV-0003.PO number is 77649657916
================================== Ai Message ==================================
Tool Calls:
  imagenow_tool (call_pbNmDDN29rHf6QS70ije5aAB)
 Call ID: call_pbNmDDN29rHf6QS70ije5aAB
  Args:
    invoice_id: INV-0003
================================= Tool Message =================================
Name: imagenow_tool

{"status": "Not Paid"}
================================== Ai Message ==================================

The PO number is an 11-digit number. Let's check the transmission status in Ivalua.
Tool Calls:
  ivalua_tool (call_2wW8wl0nHGX5sPOkBSEYbKEM)
 Call ID: call_2wW8wl0nHGX5sPOkBSEYbKEM
  Args:
    invoice_id: INV-0003
================================= Tool Message =================================
Name: ivalua_tool

{"transmission_status": "Not Transmitted", "exception_status": "MA126"}
================================== Ai Me

In [ ]:
import pandas as pd
import json
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, END

# --- Tool Definitions for Subgraph 2 ---

@tool
def imagenow_tool_subgraph2(invoice_id: str):
    """Check if invoice is present in Imagenow for Subgraph 2."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Replace with actual file path
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        queue = invoice_data.iloc[0]["Queue"]
        return {"status": "Present", "queue": queue}
    else:
        return {"status": "Not Present"}

@tool
def email_tool_subgraph2(recipient: str, message: str):
    """Send an email to the vendor or team for Subgraph 2."""
    print(f"Email forwarded to {recipient}: {message}")
    return {"email_status": "Sent"}

# List of tools for Subgraph 2
tools_subgraph2 = [imagenow_tool_subgraph2, email_tool_subgraph2]

# --- Agent State Definition for Subgraph 2 ---

class AgentState_subgraph2(TypedDict):
    """The state of the agent for Subgraph 2."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Create a mapping of tools by name for Subgraph 2
tools_by_name_subgraph2 = {tool.name: tool for tool in tools_subgraph2}

# --- Tool Node for Subgraph 2 ---

def tool_node_subgraph2(state: AgentState_subgraph2):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name_subgraph2[tool_call["name"]].invoke(tool_call["args"])
        # Convert pd.Timestamp objects to strings if necessary
        for key, value in tool_result.items():
            if isinstance(value, pd.Timestamp):
                tool_result[key] = value.isoformat()
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}

# --- Workflow Steps (Prompt) for Subgraph 2 ---

steps_prompt_subgraph2 = """
You are an AI orchestrator for the PO Invoice Processing workflow. Follow these steps to process the query:

1. Check if the invoice is present in Imagenow using the invoice ID.
2. If the invoice is not present:
   - Forward the email to the invoice processing team.
3. If the invoice is present:
   - Check in which queue it is located.
   - Create a response based on the queue.
4. At each step, use the appropriate tool and reason about the next action based on the tool's result.
"""

# --- LLM Initialization for Subgraph 2 ---

model_subgraph2 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools_subgraph2)

# --- LLM Node for Subgraph 2 ---

def call_model_subgraph2(state: AgentState_subgraph2, config: RunnableConfig):
    system_prompt = SystemMessage(
        content=f"{steps_prompt_subgraph2}\n\nYou are a workflow orchestrator for PO Invoice Processing. Decide which tool to use or provide a final answer based on the query."
    )
    # Invoke the model with the system prompt and current message history
    response = model_subgraph2.invoke([system_prompt] + state["messages"], config)
    # Append the new response to the existing messages
    state["messages"].append(response)
    return state

def should_continue_subgraph2(state: AgentState_subgraph2):
    messages = state["messages"]
    last_message = messages[-1]
    # Continue if there are pending tool calls
    if last_message.tool_calls:
        return "tools"
    return END  # Alternatively, if your framework expects a specific key, adjust accordingly

# --- Constructing the StateGraph for Subgraph 2 ---

workflow_subgraph2 = StateGraph(AgentState_subgraph2)

# Add nodes for the agent and tool processing
workflow_subgraph2.add_node("agent_subgraph2", call_model_subgraph2)
workflow_subgraph2.add_node("tools_subgraph2", tool_node_subgraph2)

# Set the entry point for Subgraph 2
workflow_subgraph2.set_entry_point("agent_subgraph2")

# Add conditional edges: if there are pending tool calls, go to the tools node; otherwise, end.
# (Adjust the key "__end__" if your framework requires a different convention.)
workflow_subgraph2.add_conditional_edges(
    "agent_subgraph2",
    should_continue_subgraph2,
    {"tools": "tools_subgraph2", "__end__": END}
)

# Add an edge from the tools node back to the agent node
workflow_subgraph2.add_edge("tools_subgraph2", "agent_subgraph2")

# Compile the workflow to produce graph2
graph2 = workflow_subgraph2.compile()


In [ ]:
# Helper function for formatting the output stream
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

# Example state for Subgraph 2 (PO Invoice for Processing)
state_subgraph2 = {
    "messages": [
        ("user", "Check invoice status in Imagenow for INV-0003 and process if required.")
    ]
}

# Execute the subgraph (graph2) and stream the output
stream_result = graph2.stream(state_subgraph2, stream_mode="values")

# Print the streamed output
print_stream(stream_result)

================================ Human Message =================================

Check invoice status in Imagenow for INV-0003 and process if required.
================================== Ai Message ==================================
Tool Calls:
  imagenow_tool_subgraph2 (call_EarFFO2r3Hk3pRjF9EmvSRdc)
 Call ID: call_EarFFO2r3Hk3pRjF9EmvSRdc
  Args:
    invoice_id: INV-0003
================================= Tool Message =================================
Name: imagenow_tool_subgraph2

{"status": "Present", "queue": "Nil"}
================================== Ai Message ==================================

The invoice INV-0003 is present in Imagenow and it is not in any queue. No further action is required at this moment.


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState. Note that we include a "messages" field so that
# when we pass the state to either subgraph (graph1 or graph2) their expected
# state (with messages) is available.
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    # Initially, "category" holds the email content. It is then overwritten
    # with the classification result.
    category: str
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Categorization function using LLM to classify the email
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState):
    # Instantiate an LLM (GPT-4) to perform categorization
    model = ChatOpenAI(model="gpt-4", temperature=0)

    # Create a system prompt instructing the LLM how to classify the email.
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )

    # Wrap the email content as a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]

    # Invoke the LLM and update the state with the classification result.
    result = model.invoke(messages)
    state["category"] = result.content.strip()
    return state

# ------------------------------------------------------------------------------
# Conditional function to route the workflow based on the email category
# ------------------------------------------------------------------------------
def categorize_decision(state: ParentState):
    if state["category"] == "Past Due Enquiry":
        return "graph1"  # Route to graph1 (Past Due Enquiry)
    elif state["category"] == "PO Invoice for Processing":
        return "graph2"  # Route to graph2 (PO Invoice for Processing)
    else:
        return END     # End the workflow if the category is not recognized

# ------------------------------------------------------------------------------
# Parent Graph Construction
# ------------------------------------------------------------------------------

# Initialize the parent StateGraph with ParentState.
parent_builder = StateGraph(ParentState)

# Add nodes:
# - "categorize_email" for classifying the email.
# - "graph1" and "graph2" are the compiled subgraphs you created earlier.
parent_builder.add_node("categorize_email", categorize_email)
parent_builder.add_node("graph1", graph1)  # graph1: Past Due Enquiry subgraph
parent_builder.add_node("graph2", graph2)  # graph2: PO Invoice Processing subgraph

# Set the entry point for the parent graph.
parent_builder.set_entry_point("categorize_email")

# Add conditional edges so that based on the classification, the workflow
# routes to the correct subgraph.
parent_builder.add_conditional_edges("categorize_email", categorize_decision, {
    "graph1": "graph1",
    "graph2": "graph2",
    END: END
})

# Compile the parent graph.
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Invocation
# ------------------------------------------------------------------------------

# When testing, ensure that the state includes an empty "messages" list.
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

# Execute the parent graph.
result = parent_graph.invoke(state_input)
print(result)


{'category': "'Past Due Enquiry'", 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': []}


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState.
# Note: The 'messages' field is needed for compatibility with your subgraphs.
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    category: str      # Initially holds the email content.
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Categorization function using LLM to classify the email.
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState):
    # Instantiate the model.
    model = ChatOpenAI(model="gpt-4", temperature=0)

    # Create a system prompt instructing the LLM how to classify the email.
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )

    # Wrap the email content (currently in state["category"]) as a HumanMessage.
    email_message = HumanMessage(content=state["category"])

    # Prepare and invoke the LLM.
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the output by stripping any extra whitespace and quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()

    # Update the state with the normalized category.
    state["category"] = result_text
    return state

# ------------------------------------------------------------------------------
# Conditional function to route the workflow based on email category.
# ------------------------------------------------------------------------------
def categorize_decision(state: ParentState):
    if state["category"] == "Past Due Enquiry":
        return "graph1"  # Route to graph1 (Past Due Enquiry)
    elif state["category"] == "PO Invoice for Processing":
        return "graph2"  # Route to graph2 (PO Invoice for Processing)
    else:
        return END     # End the workflow if the category is not recognized

# ------------------------------------------------------------------------------
# Parent Graph Construction
# ------------------------------------------------------------------------------
parent_builder = StateGraph(ParentState)

# Add nodes: the categorization node and the two subgraphs.
parent_builder.add_node("categorize_email", categorize_email)
parent_builder.add_node("graph1", graph1)  # graph1: Past Due Enquiry subgraph
parent_builder.add_node("graph2", graph2)  # graph2: PO Invoice Processing subgraph

# Set the entry point to the categorization node.
parent_builder.set_entry_point("categorize_email")

# Add conditional edges so that after categorization, the state is routed based on the email category.
parent_builder.add_conditional_edges(
    "categorize_email",
    categorize_decision,
    {"graph1": "graph1", "graph2": "graph2", END: END}
)

# Compile the parent graph.
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Execution
# ------------------------------------------------------------------------------
# Note: Ensure that the input state includes a "messages" field (even if empty).
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

# Invoke the parent graph.
result = parent_graph.invoke(state_input)
print(result)


{'category': 'Past Due Enquiry', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query**: The vendor is asking about the payment status of invoice ID 12345.\n\n**Action**: Use the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 363, 'total_tokens': 417, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9eeae247-e71a-431b-848b-7a12cae13aa7-0', usage_metadata={'input_tokens': 363, 'output_tokens': 54, 'total_tokens': 417, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState. It must include a messages field so that the subgraphs
# (which expect a messages list) get a compatible state.
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    category: str      # Initially holds the email content; later overwritten by classification.
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Node 1: Categorize the email using an LLM.
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState):
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content (currently in state["category"]) in a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)
    # Normalize the LLM output by stripping any extra quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ------------------------------------------------------------------------------
# Node 2: Route to the appropriate subgraph based on the classification.
#
# Here we explicitly call the subgraph’s invoke() method. This ensures that if the
# category is "Past Due Enquiry", then graph1 is fully executed (and similarly for graph2).
# ------------------------------------------------------------------------------
def route_to_subgraph(state: ParentState):
    if state["category"] == "Past Due Enquiry":
        # Invoke graph1 (the Past Due Enquiry workflow) and update the state.
        state = graph1.invoke(state)
    elif state["category"] == "PO Invoice for Processing":
        state = graph2.invoke(state)
    # Otherwise, leave the state unchanged (or you could handle unknown categories).
    return state

# ------------------------------------------------------------------------------
# Parent Graph Construction
# ------------------------------------------------------------------------------
parent_builder = StateGraph(ParentState)

# Add the categorization node.
parent_builder.add_node("categorize_email", categorize_email)
# Add the routing node which will call the appropriate subgraph.
parent_builder.add_node("route", route_to_subgraph)

# Create a simple linear flow: first categorize, then route.
parent_builder.add_edge("categorize_email", "route")
parent_builder.set_entry_point("categorize_email")

# Compile the parent graph.
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Execution
# ------------------------------------------------------------------------------
# Make sure the initial state includes an empty messages list.
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

# Execute the parent graph.
result = parent_graph.invoke(state_input)
print(result)


{'category': 'Past Due Enquiry', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query:**\n\nA vendor has sent an inquiry about the payment status of invoice ID 12345.\n\n**Action:**\n\nUse the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 363, 'total_tokens': 421, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78640d76-8d0e-4fd1-8a0a-afb5a54f46f7-0', usage_metadata={'input_tokens': 363, 'output_tokens': 58, 'total_tokens': 421, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState. Note that we include a "messages" field so that
# subgraphs (which expect a messages list) receive a compatible state.
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    category: str      # Initially holds the email content; later overwritten by classification.
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Node 1: Categorize the email using an LLM.
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState):
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content (currently in state["category"]) in a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the LLM output by stripping any extra quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ------------------------------------------------------------------------------
# Node 2: Route to the appropriate subgraph based on the classification.
#
# In this version, we clear the messages list (to start the subgraph afresh)
# and then repeatedly invoke the chosen subgraph until there are no pending
# tool calls.
# ------------------------------------------------------------------------------
def route_to_subgraph(state: ParentState):
    # Determine which subgraph to use and clear messages for a fresh start.
    if state["category"] == "Past Due Enquiry":
        subgraph = graph1
        state["messages"] = []
    elif state["category"] == "PO Invoice for Processing":
        subgraph = graph2
        state["messages"] = []
    else:
        return state  # Unknown category; no subgraph to run.

    # Run the chosen subgraph until it terminates.
    while True:
        state = subgraph.invoke(state)
        # The subgraph is expected to add an AIMessage that may include a 'tool_calls' attribute.
        last_msg = state["messages"][-1]
        # If there are no pending tool calls, assume the subgraph is done.
        if not hasattr(last_msg, "tool_calls") or not last_msg.tool_calls:
            break
    return state

# ------------------------------------------------------------------------------
# Parent Graph Construction
# ------------------------------------------------------------------------------
parent_builder = StateGraph(ParentState)

# Add the categorization node and the routing node.
parent_builder.add_node("categorize_email", categorize_email)
parent_builder.add_node("route", route_to_subgraph)

# Build a linear flow: first categorize, then route.
parent_builder.add_edge("categorize_email", "route")
parent_builder.set_entry_point("categorize_email")

# Compile the parent graph.
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Execution
# ------------------------------------------------------------------------------
# Ensure the initial state includes an empty "messages" list.
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

# Execute the parent graph.
result = parent_graph.invoke(state_input)
print(result)


{'category': 'Past Due Enquiry', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query:**\n\nA vendor has sent an inquiry about the payment status of invoice ID 12345.\n\n**Action:**\n\nUse the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 363, 'total_tokens': 421, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bc6b3218-6d2c-42c8-b368-63bf78a2261c-0', usage_metadata={'input_tokens': 363, 'output_tokens': 58, 'total_tokens': 421, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState.
#
# This state carries the email content (initially in the 'category' key),
# along with invoice_id, po_number, and a 'messages' list that the subgraphs
# expect.
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    category: str      # Initially holds the email content; later replaced by classification.
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Function: Categorize Email
#
# Uses an LLM to classify the email into one of the two categories.
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState) -> ParentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content as a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the output by stripping any extra quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ------------------------------------------------------------------------------
# Parent Node: Categorize and Route
#
# This node first categorizes the email, then based on the category, it resets
# the messages and calls the appropriate subgraph (graph1 or graph2).
# ------------------------------------------------------------------------------
def parent_node(state: ParentState) -> ParentState:
    # First, classify the email.
    state = categorize_email(state)

    # Depending on the classification, route to the correct subgraph.
    if state["category"] == "Past Due Enquiry":
        # Reset the messages so that graph1 (which expects a fresh message list) can build its own history.
        state["messages"] = []
        # Call graph1 (Past Due Enquiry subgraph) and update the state.
        state = graph1.invoke(state)
    elif state["category"] == "PO Invoice for Processing":
        state["messages"] = []
        state = graph2.invoke(state)
    # If the category is not recognized, you might simply return the state unchanged.
    return state

# ------------------------------------------------------------------------------
# Parent Graph Construction
#
# Here we build a parent graph that consists of a single node.
# ------------------------------------------------------------------------------
parent_builder = StateGraph(ParentState)
parent_builder.add_node("parent_node", parent_node)
parent_builder.set_entry_point("parent_node")
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Execution
#
# Make sure the initial state includes an empty messages list.
# ------------------------------------------------------------------------------
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

result = parent_graph.invoke(state_input)
print(result)


{'category': 'Check the payment status of invoice INV-0003. PO number is 77649657916', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query**: The vendor is asking about the payment status of invoice ID 12345.\n\n**Action**: Use the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 363, 'total_tokens': 417, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-275d2959-4747-4073-9e82-29bfdaab0f61-0', usage_metadata={'input_tokens': 363, 'output_tokens': 54, 'total_tokens': 417, 'input_token_details': {'audio': 0, 'cache_r

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------------------
# Define the ParentState.
#
# This state carries:
# - The email content (initially in the 'category' key, then overwritten by its classification)
# - Other fields required by the subgraphs (invoice_id, po_number)
# - A 'messages' list that the subgraphs expect
# ------------------------------------------------------------------------------
class ParentState(TypedDict):
    category: str
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ------------------------------------------------------------------------------
# Function: Categorize Email
#
# Uses an LLM to classify the email.
# ------------------------------------------------------------------------------
def categorize_email(state: ParentState) -> ParentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content as a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the output by stripping extra quotes, if any.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ------------------------------------------------------------------------------
# Helper Function: Run a Subgraph to Completion
#
# This function repeatedly calls the subgraph's invoke() method until the state's
# messages stop increasing in number. (We assume that when no new messages are added,
# the subgraph has finished its work.)
# ------------------------------------------------------------------------------
def run_subgraph_to_completion(subgraph, state: ParentState, max_iterations: int = 10) -> ParentState:
    iteration = 0
    prev_len = len(state["messages"])
    while iteration < max_iterations:
        new_state = subgraph.invoke(state)
        new_len = len(new_state["messages"])
        # If no new messages have been added, assume the subgraph is done.
        if new_len <= prev_len:
            return new_state
        state = new_state
        prev_len = new_len
        iteration += 1
    return state

# ------------------------------------------------------------------------------
# Parent Node: Categorize and Route to the Appropriate Subgraph
#
# This node first classifies the email, then clears the messages list and runs
# the appropriate subgraph (graph1 for "Past Due Enquiry", graph2 for "PO Invoice for Processing")
# to completion.
# ------------------------------------------------------------------------------
def parent_node(state: ParentState) -> ParentState:
    # Step 1: Classify the email.
    state = categorize_email(state)

    # Step 2: Route to and run the appropriate subgraph.
    if state["category"] == "Past Due Enquiry":
        state["messages"] = []  # reset messages for a fresh start
        state = run_subgraph_to_completion(graph1, state)
    elif state["category"] == "PO Invoice for Processing":
        state["messages"] = []
        state = run_subgraph_to_completion(graph2, state)
    # If the category is unrecognized, you might simply return the state as is.
    return state

# ------------------------------------------------------------------------------
# Parent Graph Construction
#
# We build a parent graph with a single node (parent_node) that does all the work.
# ------------------------------------------------------------------------------
parent_builder = StateGraph(ParentState)
parent_builder.add_node("parent_node", parent_node)
parent_builder.set_entry_point("parent_node")
parent_graph = parent_builder.compile()

# ------------------------------------------------------------------------------
# Example Input & Execution
#
# Ensure the initial state includes an empty 'messages' list.
# ------------------------------------------------------------------------------
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

result = parent_graph.invoke(state_input)
print(result)


{'category': 'Check the payment status of invoice INV-0003. PO number is 77649657916', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query:**\n\nA vendor has sent an inquiry about the payment status of invoice ID 12345.\n\n**Action:**\n\nUse the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 363, 'total_tokens': 421, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0672e418-1c7a-40dd-8684-3b5be147a8bb-0', usage_metadata={'input_tokens': 363, 'output_tokens': 58, 'total_tokens': 421, 'input_token_details': {'audio

In [ ]:
from langgraph.graph import StateGraph, START
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ==============================================================================
# Assume that graph1 and graph2 have been compiled earlier.
# For example, graph1 might be the "Past Due Enquiry" subgraph and
# graph2 might be the "PO Invoice Processing" subgraph.
# Their state schema (AgentState) is assumed to be:
#
#   {
#       "messages": List[BaseMessage]
#   }
#
# (They may use additional internal keys but for our purposes they share the key "messages".)
# ==============================================================================

# ==============================================================================
# Define the ParentState.
# This state includes keys that the parent cares about, plus a messages key.
# ==============================================================================
class ParentState(TypedDict):
    category: str      # Initially holds the email content (will be overwritten with the classification)
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ==============================================================================
# Node 1: Categorize the Email.
#
# This node uses an LLM to classify the email content (stored in state["category"])
# into one of two categories.
# ==============================================================================
def categorize_email(state: ParentState) -> ParentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content in a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the output by stripping extra quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ==============================================================================
# Node 2: Run Graph1 (Past Due Enquiry Subgraph)
#
# This node transforms the parent state to the subgraph state, calls graph1,
# and then transforms the result back to update the parent state.
# ==============================================================================
def run_graph1(state: ParentState) -> ParentState:
    # Transform ParentState to the subgraph state.
    # Our subgraph expects a state with a "messages" key.
    sub_state = {"messages": []}  # starting fresh for the subgraph
    # (If needed, you could inject extra context from the parent here.)

    # Call graph1 (which was compiled separately).
    result_sub_state = graph1.invoke(sub_state)

    # Transform back: here we update the parent's "messages" with the subgraph output.
    state["messages"] = result_sub_state["messages"]
    return state

# ==============================================================================
# Node 3: Run Graph2 (PO Invoice Processing Subgraph)
#
# This node is analogous to run_graph1 but calls graph2.
# ==============================================================================
def run_graph2(state: ParentState) -> ParentState:
    sub_state = {"messages": []}  # start fresh for graph2
    result_sub_state = graph2.invoke(sub_state)
    state["messages"] = result_sub_state["messages"]
    return state

# ==============================================================================
# Parent Node: Categorize and Route.
#
# This node first classifies the email and then—based on the classification—
# calls the appropriate subgraph via the helper nodes defined above.
# ==============================================================================
def parent_node(state: ParentState) -> ParentState:
    # Step 1: Classify the email.
    state = categorize_email(state)

    # Step 2: Depending on the classification, call the appropriate subgraph.
    if state["category"] == "Past Due Enquiry":
        state = run_graph1(state)
    elif state["category"] == "PO Invoice for Processing":
        state = run_graph2(state)
    # If the category is unrecognized, simply return the state unchanged.
    return state

# ==============================================================================
# Parent Graph Construction.
#
# Here we build the parent graph with a single node (parent_node) that does
# all the work: categorization and subgraph invocation.
# ==============================================================================
parent_builder = StateGraph(ParentState)
parent_builder.add_node("parent_node", parent_node)
parent_builder.set_entry_point("parent_node")
parent_graph = parent_builder.compile()

# ==============================================================================
# Example Input & Execution.
#
# Make sure that the input ParentState includes an empty "messages" list.
# ==============================================================================
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

result = parent_graph.invoke(state_input)
print(result)


{'category': 'Past Due Enquiry', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query:**\n\nA vendor has sent an inquiry about the payment status of invoice ID 12345.\n\n**Action:**\n\nUse the `imagenow_tool` to check the payment status of the invoice.\n\n```jsx\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 363, 'total_tokens': 421, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa21e1c5-e41b-40e5-9aa7-24f1ddf5bc50-0', usage_metadata={'input_tokens': 363, 'output_tokens': 58, 'total_tokens': 421, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [ ]:
from langgraph.graph import StateGraph, START
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ==============================================================================
# Assume that graph1 and graph2 have been compiled earlier.
#
# For example, graph1 is the "Past Due Enquiry" subgraph and graph2 is the
# "PO Invoice Processing" subgraph. Their state schema is assumed to be:
#
#   { "messages": List[BaseMessage] }
#
# (They may use additional internal keys but here the shared key is "messages".)
# ==============================================================================

# ==============================================================================
# Define the ParentState.
# This state includes keys that the parent cares about plus a messages key.
# ==============================================================================
class ParentState(TypedDict):
    category: str      # Initially holds the email content (then overwritten by classification)
    invoice_id: str
    po_number: str
    messages: List[BaseMessage]

# ==============================================================================
# Node 1: Categorize the Email.
#
# This node uses an LLM to classify the email content (initially in state["category"])
# into one of two categories.
# ==============================================================================
def categorize_email(state: ParentState) -> ParentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the categories: 'Past Due Enquiry', 'PO Invoice for Processing'."
    )
    # Wrap the email content in a HumanMessage.
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)

    # Normalize the output by stripping extra quotes.
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

# ==============================================================================
# Helper Node: Run Subgraph via Streaming
#
# This node converts the parent state to the subgraph’s expected state,
# runs the subgraph via its streaming interface to completion, and then
# merges the final subgraph state back into the parent state.
#
# (In this example, we only pass along the "messages" key.)
# ==============================================================================
def run_subgraph(subgraph, state: ParentState) -> ParentState:
    # Create a fresh subgraph state.
    sub_state = {"messages": []}
    final_state = None
    # Use the streaming interface to run the subgraph to completion.
    for chunk in subgraph.stream(sub_state):
        final_state = chunk
    # If the subgraph produced output, merge it into the parent state.
    if final_state is not None:
        state["messages"] = final_state.get("messages", [])
    else:
        state["messages"] = []
    return state

# ==============================================================================
# Parent Node: Categorize and Route.
#
# This node first categorizes the email, then—based on the classification—
# calls the appropriate subgraph via the helper node (using streaming).
# ==============================================================================
def parent_node(state: ParentState) -> ParentState:
    # Step 1: Classify the email.
    state = categorize_email(state)

    # Step 2: Depending on the classification, run the appropriate subgraph.
    if state["category"] == "Past Due Enquiry":
        state = run_subgraph(graph1, state)
    elif state["category"] == "PO Invoice for Processing":
        state = run_subgraph(graph2, state)
    # Otherwise, no subgraph is run.
    return state

# ==============================================================================
# Parent Graph Construction.
#
# We build the parent graph with a single node that does the classification
# and then calls the appropriate subgraph.
# ==============================================================================
parent_builder = StateGraph(ParentState)
parent_builder.add_node("parent_node", parent_node)
parent_builder.set_entry_point("parent_node")
parent_graph = parent_builder.compile()

# ==============================================================================
# Example Input & Execution.
#
# Make sure that the input ParentState includes an empty "messages" list.
# ==============================================================================
state_input: ParentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

result = parent_graph.invoke(state_input)
print(result)


{'category': 'Past Due Enquiry', 'invoice_id': 'INV-0003', 'po_number': '77649657916', 'messages': [AIMessage(content='**Query:**\n\nA vendor has sent an inquiry about the payment status of invoice ID 12345. Please check the status and provide the necessary details.\n\n**Action:**\n\nUse the `functions.imagenow_tool` to check the payment status in Imagenow using the invoice ID.\n\n```json\n{\n  "invoice_id": "12345"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 363, 'total_tokens': 438, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa458454-a4d1-411e-9dc9-d8968a80eee1-0', usage_metadata={'input_tokens': 363, 'output_tokens': 75, 'total_tokens': 4

In [ ]:
import pandas as pd
import json
from typing import List, Annotated, TypedDict, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

# -----------------------------------------------------------------------------
# Dummy tool decorator (if not already defined)
# -----------------------------------------------------------------------------
def tool(func):
    func.name = func.__name__
    return func

# -----------------------------------------------------------------------------
# Unified State Definition for Parent & Subgraphs
# -----------------------------------------------------------------------------
class AgentState(TypedDict):
    invoice_id: str
    po_number: str
    category: str         # Holds the email content initially and later the classification
    messages: Annotated[List[BaseMessage], add_messages]

# =============================================================================
# GRAPH 1: Past Due Enquiry Subgraph
# =============================================================================

# --- Tools for Graph 1 ---
@tool
def imagenow_tool(invoice_id: str):
    """Check payment status in Imagenow."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Update path as needed
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        payment_status = invoice_data.iloc[0]["Status"]
        return {"status": payment_status}
    else:
        return {"error": f"Invoice {invoice_id} not found."}

@tool
def lawson_tool(invoice_id: str):
    """Fetch payment details from Lawson."""
    lawson_file_path = "/content/lawson_modified.xlsx"  # Update path as needed
    lawson_df = pd.read_excel(lawson_file_path)
    invoice_data = lawson_df[lawson_df["Invoice ID"] == invoice_id]
    if not invoice_data.empty:
        payment_method = invoice_data.iloc[0]["Payment Method"]
        payment_date = invoice_data.iloc[0]["Payment Date"]
        exception_status = invoice_data.iloc[0]["Exception Status"]
        return {
            "payment_method": payment_method,
            "payment_date": payment_date,
            "exception_status": exception_status,
        }
    else:
        return {"error": f"Invoice {invoice_id} not found."}

@tool
def ivalua_tool(invoice_id: str):
    """Check transmission status in Ivalua."""
    ivalua_file_path = "/content/ivalua_dataset.xlsx"  # Update path as needed
    ivalua_df = pd.read_excel(ivalua_file_path)
    invoice_row = ivalua_df[ivalua_df["Invoice Number"] == invoice_id]
    if not invoice_row.empty:
        transmission_status = invoice_row.iloc[0]["Transmission Status"]
        exception_status = invoice_row.iloc[0]["Exception status"]
        return {
            "transmission_status": transmission_status,
            "exception_status": exception_status
        }
    else:
        return {"error": f"Invoice {invoice_id} not found in Ivalua."}

@tool
def email_tool(recipient: str, message: str):
    """Send an email."""
    # Here we simulate sending an email.
    return {"email_status": "Sent"}

tools_graph1 = [imagenow_tool, lawson_tool, ivalua_tool, email_tool]
tools_by_name = {tool.name: tool for tool in tools_graph1}

# --- Graph 1 Nodes ---
def tool_node(state: AgentState) -> AgentState:
    outputs = []
    # Assume the last message contains pending tool calls.
    last_msg = state["messages"][-1]
    for tool_call in last_msg.tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        # Convert timestamps if needed
        for key, value in tool_result.items():
            if hasattr(value, "isoformat"):
                tool_result[key] = value.isoformat()
        outputs.append(ToolMessage(
            content=json.dumps(tool_result),
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        ))
    state["messages"].extend(outputs)
    return state

steps_prompt = """
You are an AI orchestrator for the Payment Inquiry workflow.
1. Check payment status in Imagenow using invoice_id.
2. If status is "Paid": Fetch payment details from Lawson, respond to vendor, and update notes.
3. If status is "Not Paid": Determine if the PO is 10 or 11 digits; for 11-digit, check Ivalua transmission and act accordingly.
"""

model1 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools_graph1)

def call_model(state: AgentState, config) -> AgentState:
    system_prompt_msg = SystemMessage(content=steps_prompt + "\nDecide which tool to call next.")
    response = model1.invoke([system_prompt_msg] + state["messages"], config)
    state["messages"].append(response)
    return state

def should_continue(state: AgentState):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    return "end"

graph1_builder = StateGraph(AgentState)
graph1_builder.add_node("agent", call_model)
graph1_builder.add_node("tools", tool_node)
graph1_builder.set_entry_point("agent")
graph1_builder.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
graph1_builder.add_edge("tools", "agent")
graph1 = graph1_builder.compile()

# =============================================================================
# GRAPH 2: PO Invoice Processing Subgraph
# =============================================================================

# --- Tools for Graph 2 ---
@tool
def imagenow_tool_subgraph2(invoice_id: str):
    """Check if invoice is present in Imagenow for Subgraph 2."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Update path as needed
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        queue = invoice_data.iloc[0]["Queue"]
        return {"status": "Present", "queue": queue}
    else:
        return {"status": "Not Present"}

@tool
def email_tool_subgraph2(recipient: str, message: str):
    """Send an email for Subgraph 2."""
    print(f"Email forwarded to {recipient}: {message}")
    return {"email_status": "Sent"}

tools_graph2 = [imagenow_tool_subgraph2, email_tool_subgraph2]
tools_by_name_subgraph2 = {tool.name: tool for tool in tools_graph2}

# --- Graph 2 Nodes ---
def tool_node_subgraph2(state: AgentState) -> AgentState:
    outputs = []
    last_msg = state["messages"][-1]
    for tool_call in last_msg.tool_calls:
        tool_result = tools_by_name_subgraph2[tool_call["name"]].invoke(tool_call["args"])
        for key, value in tool_result.items():
            if hasattr(value, "isoformat"):
                tool_result[key] = value.isoformat()
        outputs.append(ToolMessage(
            content=json.dumps(tool_result),
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        ))
    state["messages"].extend(outputs)
    return state

steps_prompt_subgraph2 = """
You are an AI orchestrator for the PO Invoice Processing workflow.
1. Check if the invoice is present in Imagenow using invoice_id.
2. If not present, forward an email to the invoice processing team.
3. If present, check the queue and respond accordingly.
"""

model2 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools_graph2)

def call_model_subgraph2(state: AgentState, config) -> AgentState:
    system_prompt_msg = SystemMessage(content=steps_prompt_subgraph2 + "\nDecide which tool to call next.")
    response = model2.invoke([system_prompt_msg] + state["messages"], config)
    state["messages"].append(response)
    return state

def should_continue_subgraph2(state: AgentState):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools_subgraph2"
    return "end"

graph2_builder = StateGraph(AgentState)
graph2_builder.add_node("agent_subgraph2", call_model_subgraph2)
graph2_builder.add_node("tools_subgraph2", tool_node_subgraph2)
graph2_builder.set_entry_point("agent_subgraph2")
graph2_builder.add_conditional_edges("agent_subgraph2", should_continue_subgraph2, {"tools_subgraph2": "tools_subgraph2", "end": END})
graph2_builder.add_edge("tools_subgraph2", "agent_subgraph2")
graph2 = graph2_builder.compile()

# =============================================================================
# PARENT GRAPH
# =============================================================================

# Parent graph uses the unified AgentState.
# It first classifies the email (contained in state["category"]) and then routes
# to the appropriate subgraph based on that classification.
def parent_categorize(state: AgentState) -> AgentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into: 'Past Due Enquiry' or 'PO Invoice for Processing'."
    )
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

def parent_decision(state: AgentState):
    if state["category"] == "Past Due Enquiry":
        return "graph1"
    elif state["category"] == "PO Invoice for Processing":
        return "graph2"
    else:
        return END

parent_builder = StateGraph(AgentState)
parent_builder.add_node("categorize_email", parent_categorize)
# Because the state is unified, we can add the compiled subgraphs directly.
parent_builder.add_node("graph1", graph1)
parent_builder.add_node("graph2", graph2)
parent_builder.set_entry_point("categorize_email")
parent_builder.add_conditional_edges("categorize_email", parent_decision, {
    "graph1": "graph1",
    "graph2": "graph2",
    END: END
})
parent_graph = parent_builder.compile()

# =============================================================================
# EXECUTION
# =============================================================================

initial_state: AgentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

final_state = parent_graph.invoke(initial_state)
print(final_state)


{'invoice_id': 'INV-0003', 'po_number': '77649657916', 'category': 'Past Due Enquiry', 'messages': [AIMessage(content='The first tool to call would be the `imagenow_tool` to check the payment status using the invoice_id.\n\n```json\n{\n  "invoice_id": "1234567890"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 211, 'total_tokens': 253, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cacbcdce-8892-4075-91cd-b00784ed4642-0', usage_metadata={'input_tokens': 211, 'output_tokens': 42, 'total_tokens': 253, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [ ]:
import pandas as pd
import json
from typing import List, Annotated, TypedDict, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

# -----------------------------------------------------------------------------
# Dummy tool decorator (if not already provided by your framework)
# -----------------------------------------------------------------------------
def tool(func):
    func.name = func.__name__
    return func

# -----------------------------------------------------------------------------
# Unified State Definition for Parent & Subgraphs
# -----------------------------------------------------------------------------
class AgentState(TypedDict):
    invoice_id: str
    po_number: str
    category: str         # Initially holds the raw email content, then the classification
    messages: Annotated[List[BaseMessage], add_messages]

# =============================================================================
# GRAPH 1: Past Due Enquiry Subgraph
# =============================================================================

# --- Tools for Graph 1 ---
@tool
def imagenow_tool(invoice_id: str):
    """Check payment status in Imagenow."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Update path as needed
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        payment_status = invoice_data.iloc[0]["Status"]
        return {"status": payment_status}
    else:
        return {"error": f"Invoice {invoice_id} not found."}

@tool
def lawson_tool(invoice_id: str):
    """Fetch payment details from Lawson."""
    lawson_file_path = "/content/lawson_modified.xlsx"  # Update path as needed
    lawson_df = pd.read_excel(lawson_file_path)
    invoice_data = lawson_df[lawson_df["Invoice ID"] == invoice_id]
    if not invoice_data.empty:
        payment_method = invoice_data.iloc[0]["Payment Method"]
        payment_date = invoice_data.iloc[0]["Payment Date"]
        exception_status = invoice_data.iloc[0]["Exception Status"]
        return {
            "payment_method": payment_method,
            "payment_date": payment_date,
            "exception_status": exception_status,
        }
    else:
        return {"error": f"Invoice {invoice_id} not found."}

@tool
def ivalua_tool(invoice_id: str):
    """Check transmission status in Ivalua."""
    ivalua_file_path = "/content/ivalua_dataset.xlsx"  # Update path as needed
    ivalua_df = pd.read_excel(ivalua_file_path)
    invoice_row = ivalua_df[ivalua_df["Invoice Number"] == invoice_id]
    if not invoice_row.empty:
        transmission_status = invoice_row.iloc[0]["Transmission Status"]
        exception_status = invoice_row.iloc[0]["Exception status"]
        return {
            "transmission_status": transmission_status,
            "exception_status": exception_status
        }
    else:
        return {"error": f"Invoice {invoice_id} not found in Ivalua."}

@tool
def email_tool(recipient: str, message: str):
    """Send an email."""
    # Simulate sending an email.
    return {"email_status": "Sent"}

tools_graph1 = [imagenow_tool, lawson_tool, ivalua_tool, email_tool]
tools_by_name = {tool.name: tool for tool in tools_graph1}

# --- Graph 1 Nodes ---
def tool_node(state: AgentState) -> AgentState:
    outputs = []
    last_msg = state["messages"][-1]
    # Process all tool calls in the last AI message.
    for tool_call in last_msg.tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        # Convert values if necessary (e.g., timestamps)
        for key, value in tool_result.items():
            if hasattr(value, "isoformat"):
                tool_result[key] = value.isoformat()
        outputs.append(ToolMessage(
            content=json.dumps(tool_result),
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        ))
    state["messages"].extend(outputs)
    return state

steps_prompt = """
You are an AI orchestrator for the Payment Inquiry workflow.
1. Check the payment status in Imagenow using invoice_id.
2. If status is "Paid": fetch payment details from Lawson, respond to the vendor, and update notes.
3. If status is "Not Paid": determine if the PO is 10 or 11 digits; for 11-digit, check Ivalua transmission and act accordingly.
"""

model1 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools_graph1)

def call_model(state: AgentState, config) -> AgentState:
    system_prompt_msg = SystemMessage(content=steps_prompt + "\nDecide which tool to call next.")
    response = model1.invoke([system_prompt_msg] + state["messages"], config)
    state["messages"].append(response)
    return state

def should_continue(state: AgentState):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    return "end"

graph1_builder = StateGraph(AgentState)
graph1_builder.add_node("agent", call_model)
graph1_builder.add_node("tools", tool_node)
graph1_builder.set_entry_point("agent")
graph1_builder.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
graph1_builder.add_edge("tools", "agent")
graph1 = graph1_builder.compile()

# =============================================================================
# GRAPH 2: PO Invoice Processing Subgraph
# =============================================================================

# --- Tools for Graph 2 ---
@tool
def imagenow_tool_subgraph2(invoice_id: str):
    """Check if invoice is present in Imagenow for Subgraph 2."""
    imagenow_file_path = "/content/image_now_modified.xlsx"  # Update path as needed
    imagenow_df = pd.read_excel(imagenow_file_path)
    invoice_data = imagenow_df[imagenow_df["Invoice Number"] == invoice_id]
    if not invoice_data.empty:
        queue = invoice_data.iloc[0]["Queue"]
        return {"status": "Present", "queue": queue}
    else:
        return {"status": "Not Present"}

@tool
def email_tool_subgraph2(recipient: str, message: str):
    """Send an email for Subgraph 2."""
    print(f"Email forwarded to {recipient}: {message}")
    return {"email_status": "Sent"}

tools_graph2 = [imagenow_tool_subgraph2, email_tool_subgraph2]
tools_by_name_subgraph2 = {tool.name: tool for tool in tools_graph2}

# --- Graph 2 Nodes ---
def tool_node_subgraph2(state: AgentState) -> AgentState:
    outputs = []
    last_msg = state["messages"][-1]
    for tool_call in last_msg.tool_calls:
        tool_result = tools_by_name_subgraph2[tool_call["name"]].invoke(tool_call["args"])
        for key, value in tool_result.items():
            if hasattr(value, "isoformat"):
                tool_result[key] = value.isoformat()
        outputs.append(ToolMessage(
            content=json.dumps(tool_result),
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        ))
    state["messages"].extend(outputs)
    return state

steps_prompt_subgraph2 = """
You are an AI orchestrator for the PO Invoice Processing workflow.
1. Check if the invoice is present in Imagenow using invoice_id.
2. If not present, forward an email to the invoice processing team.
3. If present, check the queue and respond accordingly.
"""

model2 = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools_graph2)

def call_model_subgraph2(state: AgentState, config) -> AgentState:
    system_prompt_msg = SystemMessage(content=steps_prompt_subgraph2 + "\nDecide which tool to call next.")
    response = model2.invoke([system_prompt_msg] + state["messages"], config)
    state["messages"].append(response)
    return state

def should_continue_subgraph2(state: AgentState):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools_subgraph2"
    return "end"

graph2_builder = StateGraph(AgentState)
graph2_builder.add_node("agent_subgraph2", call_model_subgraph2)
graph2_builder.add_node("tools_subgraph2", tool_node_subgraph2)
graph2_builder.set_entry_point("agent_subgraph2")
graph2_builder.add_conditional_edges("agent_subgraph2", should_continue_subgraph2, {"tools_subgraph2": "tools_subgraph2", "end": END})
graph2_builder.add_edge("tools_subgraph2", "agent_subgraph2")
graph2 = graph2_builder.compile()

# =============================================================================
# PARENT GRAPH
# =============================================================================

# Parent node that first categorizes then uses streaming to run the appropriate subgraph.
def parent_categorize(state: AgentState) -> AgentState:
    model = ChatOpenAI(model="gpt-4", temperature=0)
    system_prompt = SystemMessage(
        content="Classify the email into one of the following: 'Past Due Enquiry' or 'PO Invoice for Processing'."
    )
    email_message = HumanMessage(content=state["category"])
    messages = [system_prompt, email_message]
    result = model.invoke(messages)
    result_text = result.content.strip()
    if ((result_text.startswith("'") and result_text.endswith("'")) or
        (result_text.startswith('"') and result_text.endswith('"'))):
        result_text = result_text[1:-1].strip()
    state["category"] = result_text
    return state

def parent_node(state: AgentState) -> AgentState:
    # First, classify the email.
    state = parent_categorize(state)
    # Based on classification, run the corresponding subgraph via streaming.
    if state["category"] == "Past Due Enquiry":
        final_state = None
        for chunk in graph1.stream(state):
            final_state = chunk
        return final_state if final_state is not None else state
    elif state["category"] == "PO Invoice for Processing":
        final_state = None
        for chunk in graph2.stream(state):
            final_state = chunk
        return final_state if final_state is not None else state
    return state

parent_builder = StateGraph(AgentState)
parent_builder.add_node("parent_node", parent_node)
parent_builder.set_entry_point("parent_node")
parent_graph = parent_builder.compile()

# =============================================================================
# EXECUTION
# =============================================================================

initial_state: AgentState = {
    "category": "Check the payment status of invoice INV-0003. PO number is 77649657916",
    "invoice_id": "INV-0003",
    "po_number": "77649657916",
    "messages": []
}

final_state = parent_graph.invoke(initial_state)
print(final_state)


{'invoice_id': 'INV-0003', 'po_number': '77649657916', 'category': 'Past Due Enquiry', 'messages': [AIMessage(content='The first tool to call would be the `imagenow_tool` to check the payment status using the invoice_id.\n\n```json\n{\n  "invoice_id": "1234567890"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 213, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e0b11136-5e27-4f58-9ea7-7c8f98a712c2-0', usage_metadata={'input_tokens': 213, 'output_tokens': 42, 'total_tokens': 255, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}
